<a href="https://colab.research.google.com/github/ITDeema/DataScience/blob/main/DataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!apt-get update -y
!apt-get install -y chromium-chromedriver
!pip install -q selenium pandas openpyxl


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 3s (1,535 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest versio

In [17]:
!pip install google-search-results

In [3]:
from serpapi import GoogleSearch
import pandas as pd
import time

api_key = "783dcf17ea65c6a84a44cceb4fd71c5129f14f0eefc1d4cfc5c3249f8a095bd4"

queries = [
    # Makkah
    "Hotels in Makkah",
    "Hotels near Masjid al Haram",
    "Hotels in Ajyad Makkah",
    "Hotels in Aziziyah Makkah",
    "Hotels in Al Misfalah Makkah",
    "Hotels in Al Shubaikah Makkah",
    "Hotels in Al Naseem Makkah",
    "Hotels in Kudai Makkah",
    "Hotels in Jarwal Makkah",

    # Madinah
    "Hotels in Madinah",
    "Hotels near Al Masjid an Nabawi",
    "Hotels in Central Madinah",
    "Hotels in Al Haram Madinah",
    "Hotels in Quba Madinah",
    "Hotels in Al Azhari Madinah",
    "Hotels in Al Aridh Madinah",
    "Hotels in Bani Khidrah Madinah"
]

date_ranges = [
    ("2026-02-15", "2026-02-20"),
    ("2026-03-10", "2026-03-15"),
    ("2026-03-20", "2026-03-25"),
    ("2026-05-20", "2026-05-25"),
    ("2026-06-10", "2026-06-15"),
    ("2026-07-01", "2026-07-06")
]

all_hotels = []

for checkin, checkout in date_ranges:
    for q in queries:
        for start in [0, 20, 40, 60, 80]:

            params = {
                "engine": "google_hotels",
                "q": q,
                "check_in_date": checkin,
                "check_out_date": checkout,
                "adults": 2,
                "currency": "SAR",
                "gl": "sa",
                "hl": "ar",
                "start": start,
                "api_key": api_key
            }

            search = GoogleSearch(params)
            results = search.get_dict()

            for h in results.get("properties", []):


                if checkin.startswith("2026-03"):
                    season_type = "Ramadan Season"
                    context_text = "High spiritual tourism season during Ramadan"
                elif checkin.startswith("2026-06"):
                    season_type = "Hajj Peak"
                    context_text = "Peak pilgrimage demand period"
                elif checkin.startswith("2026-05"):
                    season_type = "Pre-Hajj"
                    context_text = "Pre-Hajj booking increase period"
                else:
                    season_type = "Regular Umrah"
                    context_text = "Regular travel season with moderate hotel demand"

                address = h.get("address")
                if not address:
                    gps = h.get("gps_coordinates", {})
                    lat = gps.get("latitude")
                    lng = gps.get("longitude")
                    if lat and lng:
                        address = f"Lat: {lat}, Lng: {lng}"
                    else:
                        address = None


                city = "Madinah" if "Madinah" in q else "Makkah"

                all_hotels.append({
                    "Hotel Name": h.get("name"),
                    "Price (SAR)": h.get("rate_per_night", {}).get("lowest"),
                    "Rating": h.get("overall_rating"),
                    "Reviews": h.get("reviews"),
                    "Address": address,
                    "City": city,
                    "Check-in": checkin,
                    "Check-out": checkout,
                    "Search Area": q,
                    "Season_Type": season_type,
                    "Religious_Context": context_text
                })

            time.sleep(1)

df_final = pd.DataFrame(all_hotels)

df_final = df_final.drop_duplicates(
    subset=["Hotel Name", "Address", "Check-in"]
).reset_index(drop=True)

print("Total rows collected:", len(df_final))

df_final.to_csv("makkah_madinah_hajj_umrah_2026_with_unstructured.csv", index=False)

df_final.head()


Total rows collected: 1139


,Hotel Name,Price (SAR),Rating,Reviews,Address,City,Check-in,Check-out,Search Area,Season_Type,Religious_Context
0,فندق ميلينيوم مكة النسيم,‏٩٢٢ ر.س.‏,4.3,13406.0,"Lat: 21.380888499999998, Lng: 39.8720317",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
1,فندق سنود الدانه | Snood Al-Dana Hotel,‏٣٠٤ ر.س.‏,3.9,935.0,"Lat: 21.3997575, Lng: 39.88258090000001",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
2,دبل ترى باي هيلتون جبل عمر مكة,‏٣٬٢٥٨ ر.س.‏,4.5,8755.0,"Lat: 21.4231692, Lng: 39.8178141",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
3,فندق جواهر الهاشمية 4,‏٢٥٩ ر.س.‏,3.1,46.0,"Lat: 21.4027933, Lng: 39.8767989",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
4,فندق ذوق الخيال,‏٣٦٥ ر.س.‏,3.9,355.0,"Lat: 21.4125479, Lng: 39.86540300000001",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan


In [4]:
from serpapi import GoogleSearch
import pandas as pd
import time

api_key = "783dcf17ea65c6a84a44cceb4fd71c5129f14f0eefc1d4cfc5c3249f8a095bd4"

queries = [
    "Hotels in Taif",
    "Hotels in Taif for Hajj visitors",
    "Hotels in Taif near Miqat",
    "Umrah visitor hotels in Taif",
    "Budget hotels in Taif for pilgrims",
    "Luxury hotels in Taif during Hajj season",
    "Hotels in Taif near Al Sail Al Kabir Miqat"
]

date_ranges = [
    ("2026-02-15", "2026-02-20"),
    ("2026-03-10", "2026-03-15"),
    ("2026-03-20", "2026-03-25"),
    ("2026-05-20", "2026-05-25"),
    ("2026-06-10", "2026-06-15"),
    ("2026-07-01", "2026-07-06")
]

all_hotels = []

for checkin, checkout in date_ranges:
    for q in queries:
        for start in [0, 20, 40]:

            params = {
                "engine": "google_hotels",
                "q": q,
                "check_in_date": checkin,
                "check_out_date": checkout,
                "adults": 2,
                "currency": "SAR",
                "gl": "sa",
                "hl": "ar",
                "start": start,
                "api_key": api_key
            }

            search = GoogleSearch(params)
            results = search.get_dict()

            for h in results.get("properties", []):


                if checkin.startswith("2026-03"):
                    season_type = "Ramadan Season"
                    context_text = "Ramadan spiritual tourism season affecting Taif hotels"
                elif checkin.startswith("2026-06"):
                    season_type = "Hajj Peak"
                    context_text = "Hajj season overflow demand with pilgrims staying in Taif"
                elif checkin.startswith("2026-05"):
                    season_type = "Pre-Hajj"
                    context_text = "Pre-Hajj increasing accommodation demand in Taif"
                else:
                    season_type = "Regular Umrah"
                    context_text = "Regular Umrah related travel near Makkah region"

                address = h.get("address")
                if not address:
                    gps = h.get("gps_coordinates", {})
                    lat = gps.get("latitude")
                    lng = gps.get("longitude")
                    if lat and lng:
                        address = f"Lat: {lat}, Lng: {lng}"
                    else:
                        address = None

                all_hotels.append({
                    "Hotel Name": h.get("name"),
                    "City": "Taif",
                    "Price (SAR)": h.get("rate_per_night", {}).get("lowest"),
                    "Rating": h.get("overall_rating"),
                    "Reviews": h.get("reviews"),
                    "Address": address,
                    "Check-in": checkin,
                    "Check-out": checkout,
                    "Search Area": q,
                    "Season_Type": season_type,
                    "Religious_Context": context_text
                })

            time.sleep(1)

df_taif = pd.DataFrame(all_hotels)

df_taif = df_taif.drop_duplicates(
    subset=["Hotel Name", "Address", "Check-in"]
).reset_index(drop=True)

print("Total Taif rows:", len(df_taif))

df_taif.to_csv("taif_hajj_umrah_2026.csv", index=False)

df_taif.head()

Total Taif rows: 341


,Hotel Name,City,Price (SAR),Rating,Reviews,Address,Check-in,Check-out,Search Area,Season_Type,Religious_Context
0,الديوان النجدي للشقق المخدومة اقتصادي - One-Be...,Taif,‏١٧٩ ر.س.‏,NaN,NaN,"Lat: 21.484140396118164, Lng: 40.48775863647461",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...
1,ناضرة الوسام للشقق المخدومة - Two-Bedroom Apar...,Taif,‏١٦٢ ر.س.‏,2.800000,7.0,"Lat: 21.22488021850586, Lng: 40.38262176513672",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...
2,فخامة الديار للشقق المخدومة Fakhamat Aldyar Fo...,Taif,‏١٩٣ ر.س.‏,4.013043,23.0,"Lat: 21.319499969482422, Lng: 40.428550720214844",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...
3,شاليهات تولين - Two-Bedroom Apartment,Taif,‏٣٠٠ ر.س.‏,NaN,NaN,"Lat: 21.49803924560547, Lng: 40.4958610534668",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...
4,شاليهات دشيل لبيوت العطلات الاقتصادية - Two-Be...,Taif,‏٤٤٧ ر.س.‏,4.950000,8.0,"Lat: 21.230449676513672, Lng: 40.38397979736328",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...


In [5]:
import pandas as pd

df_main = pd.read_csv("makkah_madinah_hajj_umrah_2026_with_unstructured.csv")
df_taif = pd.read_csv("taif_hajj_umrah_2026.csv")

print("Rows before merge:")
print("Main file:", len(df_main))
print("Taif file:", len(df_taif))

df_main.columns = df_main.columns.str.strip()
df_taif.columns = df_taif.columns.str.strip()

df_combined = pd.concat([df_main, df_taif], ignore_index=True, sort=False)

df_combined = df_combined.drop_duplicates(
    subset=["Hotel Name", "Address", "Check-in"]
).reset_index(drop=True)

print("Final merged rows:", len(df_combined))

df_combined.to_csv("FINAL_HAJJ_UMRAH_ALL_CITIES_2026.csv", index=False)

df_combined.head()


Rows before merge:
Main file: 1139
Taif file: 341
Final merged rows: 1480


,Hotel Name,Price (SAR),Rating,Reviews,Address,City,Check-in,Check-out,Search Area,Season_Type,Religious_Context
0,فندق ميلينيوم مكة النسيم,‏٩٢٢ ر.س.‏,4.3,13406.0,"Lat: 21.380888499999998, Lng: 39.8720317",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
1,فندق سنود الدانه | Snood Al-Dana Hotel,‏٣٠٤ ر.س.‏,3.9,935.0,"Lat: 21.3997575, Lng: 39.88258090000001",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
2,دبل ترى باي هيلتون جبل عمر مكة,‏٣٬٢٥٨ ر.س.‏,4.5,8755.0,"Lat: 21.4231692, Lng: 39.8178141",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
3,فندق جواهر الهاشمية 4,‏٢٥٩ ر.س.‏,3.1,46.0,"Lat: 21.4027933, Lng: 39.8767989",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
4,فندق ذوق الخيال,‏٣٦٥ ر.س.‏,3.9,355.0,"Lat: 21.4125479, Lng: 39.86540300000001",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan


In [6]:
import numpy as np

df_raw = pd.read_csv("FINAL_HAJJ_UMRAH_ALL_CITIES_2026.csv")

# إنشاء نسخة للعمل عليها
df = df_raw.copy()

In [7]:
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1480 entries, 0 to 1479
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Hotel Name         1480 non-null   object 
 1   Price (SAR)        796 non-null    object 
 2   Rating             1400 non-null   float64
 3   Reviews            1400 non-null   float64
 4   Address            1480 non-null   object 
 5   City               1480 non-null   object 
 6   Check-in           1480 non-null   object 
 7   Check-out          1480 non-null   object 
 8   Search Area        1480 non-null   object 
 9   Season_Type        1480 non-null   object 
 10  Religious_Context  1480 non-null   object 
dtypes: float64(2), object(9)
memory usage: 127.3+ KB


,Rating,Reviews
count,1400.000000,1400.000000
mean,4.098302,3588.117857
std,0.463801,9967.885672
min,2.000000,1.000000
25%,3.800000,157.000000
50%,4.100000,577.500000
75%,4.400000,1942.000000
max,5.000000,117056.000000


In [8]:
missing_table = pd.DataFrame({
    "Missing Count": df.isnull().sum(),
    "Missing %": df.isnull().mean() * 100
})

missing_table.sort_values("Missing %", ascending=False)

,Missing Count,Missing %
Price (SAR),684,46.216216
Rating,80,5.405405
Reviews,80,5.405405
Hotel Name,0,0.000000
Address,0,0.000000
City,0,0.000000
Check-in,0,0.000000
Check-out,0,0.000000
Search Area,0,0.000000
Season_Type,0,0.000000


In [9]:
# فحص عدد التكرارات بناءً على الموقع وتاريخ الدخول بدلاً من الاسم
duplicate_count = df.duplicated(subset=['Address', 'Check-in', 'Check-out']).sum()
print(f"Number of duplicates found: {duplicate_count}")

# حذف التكرارات
df.drop_duplicates(subset=['Address', 'Check-in', 'Check-out'], inplace=True)
df.drop_duplicates(inplace=True)

Number of duplicates found: 0


In [10]:
import re

def arabic_to_english_number(text):
    arabic_numbers = "٠١٢٣٤٥٦٧٨٩"
    english_numbers = "0123456789"
    translator = str.maketrans(arabic_numbers, english_numbers)
    return text.translate(translator)

def clean_price(value):
    if pd.isna(value):
        return None
    value = str(value)
    value = arabic_to_english_number(value)
    value = re.sub(r"[^\d]", "", value)   # ← هذا هو التغيير
    if value == "":
        return None
    return float(value)

df["Price (SAR)"] = df["Price (SAR)"].apply(clean_price)

print(df["Price (SAR)"])

0        922.0
1        304.0
2       3258.0
3        259.0
4        365.0
         ...  
1475       NaN
1476       NaN
1477       NaN
1478       NaN
1479    1136.0
Name: Price (SAR), Length: 1480, dtype: float64


In [11]:
df["Price (SAR)"] = df["Price (SAR)"].fillna(df["Price (SAR)"].median())
df["Rating"] = df["Rating"].fillna(df["Rating"].mean()).round(1)
df["Reviews"] = df["Reviews"].fillna(0)

print(df.isnull().sum())

Hotel Name           0
Price (SAR)          0
Rating               0
Reviews              0
Address              0
City                 0
Check-in             0
Check-out            0
Search Area          0
Season_Type          0
Religious_Context    0
dtype: int64


In [12]:
# 1. استخراج الإحداثيات مباشرة في الأعمدة الجديدة
# أضفنا علامة -? تحسباً لو كان هناك إحداثيات بالسالب (أفضل ممارسة في علم البيانات)
df[["Latitude", "Longitude"]] = df["Address"].str.extract(r"Lat:\s*(-?[\d\.]+),\s*Lng:\s*(-?[\d\.]+)")

# 2. تحويل الأعمدة إلى أرقام عشرية
df["Latitude"] = pd.to_numeric(df["Latitude"])
df["Longitude"] = pd.to_numeric(df["Longitude"])

# 3. التأكد من النتيجة
print(df[["Address", "Latitude", "Longitude"]].head())

                                    Address   Latitude  Longitude
0  Lat: 21.380888499999998, Lng: 39.8720317  21.380889  39.872032
1   Lat: 21.3997575, Lng: 39.88258090000001  21.399757  39.882581
2          Lat: 21.4231692, Lng: 39.8178141  21.423169  39.817814
3          Lat: 21.4027933, Lng: 39.8767989  21.402793  39.876799
4   Lat: 21.4125479, Lng: 39.86540300000001  21.412548  39.865403


In [13]:
df["Check-in"] = pd.to_datetime(df["Check-in"])
df["Check-out"] = pd.to_datetime(df["Check-out"])
print(df[["Check-in", "Check-out"]])

       Check-in  Check-out
0    2026-03-10 2026-03-15
1    2026-03-10 2026-03-15
2    2026-03-10 2026-03-15
3    2026-03-10 2026-03-15
4    2026-03-10 2026-03-15
...         ...        ...
1475 2026-07-01 2026-07-06
1476 2026-07-01 2026-07-06
1477 2026-07-01 2026-07-06
1478 2026-07-01 2026-07-06
1479 2026-07-01 2026-07-06

[1480 rows x 2 columns]


In [14]:
df.info()
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1480 entries, 0 to 1479
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Hotel Name         1480 non-null   object        
 1   Price (SAR)        1480 non-null   float64       
 2   Rating             1480 non-null   float64       
 3   Reviews            1480 non-null   float64       
 4   Address            1480 non-null   object        
 5   City               1480 non-null   object        
 6   Check-in           1480 non-null   datetime64[ns]
 7   Check-out          1480 non-null   datetime64[ns]
 8   Search Area        1480 non-null   object        
 9   Season_Type        1480 non-null   object        
 10  Religious_Context  1480 non-null   object        
 11  Latitude           1480 non-null   float64       
 12  Longitude          1480 non-null   float64       
dtypes: datetime64[ns](2), float64(5), object(6)
memory usage: 150.4

,0
Hotel Name,0
Price (SAR),0
Rating,0
Reviews,0
Address,0
City,0
Check-in,0
Check-out,0
Search Area,0
Season_Type,0


In [15]:
df.to_csv("CLEANED_HAJJ_UMRAH.csv", index=False)

In [18]:
# إنشاء مدة الإقامة
df["Stay_Duration"] = (df["Check-out"] - df["Check-in"]).dt.days

# إنشاء فئة سعرية
df["Price_Category"] = pd.cut(
    df["Price (SAR)"],
    bins=[0, 300, 800, 1500, 10000],
    labels=["Budget", "Mid-Range", "Premium", "Luxury"]
)

In [19]:
# One-Hot Encoding
df_encoded = pd.get_dummies(
    df,
    columns=["City", "Season_Type", "Price_Category"],
    drop_first=True
)

In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

numeric_features = ["Price (SAR)", "Rating", "Reviews", "Stay_Duration"]

df_encoded[numeric_features] = scaler.fit_transform(df_encoded[numeric_features])

In [21]:
df_encoded.to_csv("PROCESSED_HAJJ_UMRAH_2026.csv", index=False)